## Solution Approach:

### Used Input Dataset:
For the chosen Use Case analysis the kaggle input dataset was used:
https://www.kaggle.com/datasets/kaushal2896/airplane-accidents-severity-dataset 

The dataset consists of two files, both shared in the same folder:

●	Train.csv: [10000 x 12 excluding the headers] contains Training data

●	Test.csv: [2500 x 11 excluding the headers] contains Test data

#### Columns Description
Accident_ID	unique id assigned to each row

Accident_Type_Code	the type of accident (factor, not numeric)

Cabin_Temperature	the last recorded temperature before the incident, measured in degrees Fahrenheit

Turbulence_In_gforces	the recorded/estimated turbulence experienced during the accident

Control_Metric	an estimation of how much control the pilot had during the incident given the factors at play

Total_Safety_Complaints	number of complaints from mechanics prior to the accident

Days_Since_Inspection	how long the plane went without inspection before the incident

Safety_Score	a measure of how safe the plane was deemed to be

Violations	number of violations that the aircraft received during inspections

Severity	a description (4 level factor) on the severity of the crash [Target]

#### Libraries Used:
•	Pandas for data manipulation
•	NumPy for performing mathematical operations on the data
•	Matplotlib, Seaborn and Scikitplot for visualization
•	Sklearn for pre-processing and model building & evaluation
#### Steps and Inferences:
1.	Data was read and loaded into a DataFrame and first few records were visualized
2.	The dataTypes were understood and we could infer the following
a.	Safety_Score, Control_Metric, Turbulence_In_gforces, Cabin_Temprature, Max_Elevation and Adverse_Weather_Metric are continuous in nature
b.	Severity, Days_Since_Inspection, Total_Safety_Compliants, Accident_Type_Code and Violations are Discreet categorical data
3.	There were no null values present in the dataset
4.	Five point summary was performed on the continuous data and following were observed
a.	Mean of the following attributes are not in sync with the median which implies the presence of outliers
i.	Total_Safety_Complaints
ii.	Violations
iii.	Adverse_Weather_Metric
5.	Exploratory Data Analysis was performed starting with univariate analysis and following were observed
a.	Safety score is normally distributed  and has few outliers
b.	Control_Metric is skewed to the left but normally distributed with outliers
c.	Turbulence_In_gForce is skewed to the left but normally distributed with outliers
d.	Cabin_Temprature is skewed to the right but normally distributed with outliers
e.	Max_Elevation is t normally distributed with outliers
f.	Adverse_Weather_Metric is highly skewed to the right with huge number of outliers
g.	Average Days_Since_Inspection is 13
h.	number of records with Total_Safety_Compliants as 0-10 is high
i.	Accident_Type_Code 5 has fewer number of records
j.	2 violations has the highest number of records
k.	There is a slight class imbalance with 'Significant_Damage_And_Fatalities' group having fewer number of records
6.	Multivariate analysis were performed along with correlation matrix and below were the features with top correlations
Accident_Type_Code  	Adverse_Weather_Metric    	0.739361
Safety_Score        	Days_Since_Inspection     	0.685386
Control_Metric      	Turbulence_In_gforces     	0.643285
7.	Numerical data was standardized using z-score
8.	Removed all records with z-score greater and lesser than 3 and -3 respectively as the values are outliers
9.	493 records were removed as they were considered outliers
10.	Label Encoding was performed on the Target Column
11.	Basic Feature Engineering was performed in the interest of time and lack of domain expertise to create new features
12.	Independent features and Target columns were split as X and Y
13.	K-Fold cross-validation was performed to create multiple folds of  train and validation data
14.	2 Models were taken into consideration based on the data
a.	Gradient Boosting
b.	XG Boost
15.	XGBoost was considered as final model and RandomSearchCV was used for hyper-parameterization
16.	Validation set was evaluated and all metrics (F1, recall, precision and accuracy) was above 95%
17.	Test Set was also loaded and predicted with the model already built

### Importing Necessary Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler 
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix
import scikitplot as skplt

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### Reading and understanding the data-set

In [ ]:
data=pd.read_csv('/kaggle/input/airplane-accidents-severity-dataset/train.csv')

#### Checking the head of the data-set

In [ ]:
data.head()

##### ● Acciedent_ID : This is the unique ID of the accident as recorded by the authorities
##### ● Adverse_Weather_Metric : measured weather metric in the basis of the adverse events occured
##### ● Violations: number of violations that the aircraft received during inspections
##### ● Max_Elevation : maximim altitude the airplane has reached during the event
##### ● Accident_Type_Code : Code of the accident classified by the authorities
##### ● Cabin_Temperature: the last recorded temperature before the incident, measured in degrees Fahrenheit
##### ● Turbulence_In_gforces :the recorded/estimated turbulence experienced during the accident
##### ● Control_Metric : an estimation of how much control the pilot had during the incident given the factors at play
##### ● Total_Safety_Complaints:number of complaints from mechanics prior to the accident
##### ● Days_Since_Inspection:how long the plane went without inspection before the incident
##### ● Safety_Score: a measure of how safe the plane was deemed to be

#### Checking the data-types of the data

In [ ]:
data.dtypes

##### Safety_Score, Control_Metric,Turbulence_In_gforces,Cabin_Temprature,Max_Elevation,Adverse_Weather_Metric are continous in nature
##### Severity, Days_Since_Inspection, Total_Safety_Compliants, Accident_Type_Code and Violations are Discreet categorical data

#### Checking the information of the data

In [ ]:
data.info()

In [ ]:
data.shape

##### There are 10000 records with 11 independent variables and 1 target variable

#### Check for any null values in the data

In [ ]:
na_values=data.isna().sum()
print(na_values)

#### To describe the data- Five point summary- Remove Accident ID as it does not help with analysis of data

In [ ]:
data.drop(['Accident_ID'],axis=1,inplace=True)
data.describe().T

##### Mean of the following attributes are not in sync with the median which implies the presence of outliers
###### Total_Safety_Complaints, Violations, Adverse_Weather_Metric

### Exploratory Data Analytics

#### Uni-Variate Analysis

In [ ]:
# Distribution of continous data

# Safety_Score, Control_Metric,Turbulence_In_gforce


plt.figure(figsize=(30,6))

#Subplot 1
plt.subplot(1,3,1)
plt.title('Safety_Score')
sns.distplot(data['Safety_Score'],color='red')

#Subplot 2
plt.subplot(1,3,2)
plt.title('Control_Metric')
sns.distplot(data['Control_Metric'],color='blue')

#Subplot 3
plt.subplot(1,3,3)
plt.title('Turbulence_In_gforces')
sns.distplot(data['Turbulence_In_gforces'],color='green')



plt.figure(figsize=(30,6))

#Subplot 1- Boxplot
plt.subplot(1,3,1)
plt.title('Safety_Score')
sns.boxplot(data['Safety_Score'],orient='horizondal',color='red')

#Subplot 2
plt.subplot(1,3,2)
plt.title('Control_Metric')
sns.boxplot(data['Control_Metric'],orient='horizondal',color='blue')

#Subplot 3
plt.subplot(1,3,3)
plt.title('Turbulence_In_gforces')
sns.boxplot(data['Turbulence_In_gforces'],orient='horizondal',color='green')


##### Safety score is normally distributed  and has few outliers
##### Control_Metric is skewed to the left but normally distributed with outliers
##### Turbulence_In_gForce is skewed to the left but normally distributed with outliers 

In [ ]:
# Distribution of continous data
# Cabin_Temprature, Max_Elevation, Adverse_Weather_Metric

plt.figure(figsize=(30,6))

#Subplot 1
plt.subplot(1,3,1)
plt.title('Cabin_Temperature')
sns.distplot(data['Cabin_Temperature'],color='red')

#Subplot 2
plt.subplot(1,3,2)
plt.title('Max_Elevation')
sns.distplot(data['Max_Elevation'],color='blue')

#Subplot 3
plt.subplot(1,3,3)
plt.title('Adverse_Weather_Metric')
sns.distplot(data['Adverse_Weather_Metric'],color='green')



plt.figure(figsize=(30,6))

#Subplot 1- Boxplot
plt.subplot(1,3,1)
plt.title('Cabin_Temperature')
sns.boxplot(data['Cabin_Temperature'],orient='horizondal',color='red')

#Subplot 2
plt.subplot(1,3,2)
plt.title('Max_Elevation')
sns.boxplot(data['Max_Elevation'],orient='horizondal',color='blue')

#Subplot 3
plt.subplot(1,3,3)
plt.title('Adverse_Weather_Metric')
sns.boxplot(data['Adverse_Weather_Metric'],orient='horizondal',color='green')


##### Cabin_Temprature is skewed to the right but normally distributed with outliers
##### Max_Elevation is t normally distributed with outliers
##### Adverse_Weather_Metric is highly skewed to the right with huge number of outliers 

In [ ]:
##### Days_Since_Inspection, Total_Safety_Compliant
plt.figure(figsize=(30,6))

#Subplot 1
plt.subplot(1,2,1)
plt.title('Days_Since_Inspection')
sns.countplot(data['Days_Since_Inspection'],color='red')

#Subplot 2
plt.subplot(1,2,2)
plt.title('Total_Safety_Complaints')
sns.countplot(data['Total_Safety_Complaints'],color='blue')


##### Average Days_Since_Inspection is 13
##### number of records with Total_Safety_Compliants as 0-10 is high

In [ ]:
# Accident_Type_Code and Violations

plt.figure(figsize=(30,6))


#Subplot 1
plt.subplot(1,2,1)
plt.title('Accident_Type_Code')
sns.countplot(data['Accident_Type_Code'],color='red')

#Subplot 2
plt.subplot(1,2,2)
plt.title('Violations')
sns.countplot(data['Violations'],color='blue')

#####  Accident_Type_Code 5 has fewer number of records
##### 2 violations has the highest number of records

In [ ]:
plt.figure(figsize=(30,6))

plt.title('Severity')
sns.countplot(data['Severity'],color='red')

#### There is a slight class imbalance with 'Significant_Damage_And_Fatalities' group having fewer number of records

#### Multi-variate Analysis

In [ ]:
sns.pairplot(data,palette="Set2", diag_kind="kde", height=2.5)

#### Correlation Matrix

In [ ]:
correlation=data.corr()
correlation.style.background_gradient(cmap='coolwarm')

In [ ]:
data.corr()>0.5

In [ ]:
data.corr()<-0.5

In [ ]:
df=data.drop(['Severity'],axis=1)
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=20):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

print("Top Absolute Correlations")
print(get_top_abs_correlations(df, 3))

##### These pairs of independent attibutes have good correlation

#### Handle Outliers

In [ ]:
data.info()

In [ ]:
dataNumericals = pd.DataFrame(data, columns =data.columns[data.dtypes == 'float64']) 
dataNumericals.head()

#### Applying z-score to scale the data and standardize the data


In [ ]:
dataNumericals=dataNumericals.apply(zscore)

In [ ]:
dataNumericals.head()

#### Removing all records with z-score greater and lesser than 3 and -3 respectivley as the values are outliers

In [ ]:
floats = dataNumericals.columns[dataNumericals.dtypes == 'float64']
for columns in floats:
    indexNames_larger = dataNumericals[dataNumericals[columns]>3].index
    indexNames_lesser = dataNumericals[dataNumericals[columns]<-3].index
    # Delete these row indexes from dataFrame
    dataNumericals.drop(indexNames_larger , inplace=True)
    dataNumericals.drop(indexNames_lesser , inplace=True)
    data.drop(indexNames_larger , inplace=True)
    data.drop(indexNames_lesser , inplace=True)

In [ ]:
dataNumericals.info()

In [ ]:
data.info()

#### 493 records were removed as they were considered outliers

#### Merging the scaled columns back to the original dataframe

In [ ]:
data.drop(data.columns[data.dtypes == 'float64'],axis=1,inplace=True)

In [ ]:
data.head()

In [ ]:
for column in dataNumericals.columns:
    data[column]=dataNumericals[column]

In [ ]:
data.head()

In [ ]:
data.info()

#### Label Encoding the Target Column

In [ ]:
data['Severity'].unique()

In [ ]:
encoder=LabelEncoder()
data['Severity']=encoder.fit_transform(data['Severity'])

In [ ]:
data.head()

### Feature Engineering

In [ ]:
data['Total_Safety_Complaints'] = np.power(2, data['Total_Safety_Complaints'])
data['Days_Since_Inspection'] = np.power(2, data['Days_Since_Inspection'])
data['Safety_Score'] = np.power(2, data['Safety_Score'])

#### Splitting X-independent attributes and Y-dependent attributes and keeping the test set seperate
#### Creating multiple cross-validation to reduce overfitting

In [ ]:
X=data.drop(['Severity'],axis=1)

In [ ]:
Y=data['Severity']

In [ ]:
Xtrain_val,X_test,ytrain_val,Y_test=train_test_split(X,Y,test_size=0.2,random_state=22)

In [ ]:
kf = KFold(n_splits=10,random_state=2,shuffle=True)
kf.get_n_splits(Xtrain_val)
print(kf)


for train_index, val_index in kf.split(Xtrain_val):
    print("TRAIN:", train_index, "VALIDATION:", val_index)
    X_train, X_val = Xtrain_val.iloc[train_index], Xtrain_val.iloc[val_index]
    y_train, y_val = ytrain_val.iloc[train_index], ytrain_val.iloc[val_index]